In [1]:
# Logistic regression learning algorithm using TensorFlow library.
# Base example from Aymeric Damien, https://github.com/aymericdamien/TensorFlow-Examples/

# Currently a loose idea of what we need to make a model for 
# Still need to read in data
# MNIST data kept in comments just in case we need to retest something

In [14]:
import tensorflow as tf
import numpy as np
import json
import os
from tensorflow.python.platform import gfile
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [15]:
# read in data from chosen files
data = []
labels = []
# correct user is specified for which labels will be marked as '1'
correctUser = "hannah"
path = './data'

# read in data from files
for filename in os.listdir(path):
    fp = open("./data/"+filename, "r")
    j = json.load(fp)

    # after file is open put the json data into lists for training
    for user in j:
        if user == correctUser:
            for attempt in j[user]:
                templist = []
                for key in attempt["Keys"]:
                    templist.append(key["Fight"])
                    templist.append(key["Dwell"])
                templist.append(attempt["Metadata"][0]["Total Time"])

                data.append(templist)
                labels.append(1)
        else:
            for attempt in j[user]:
                templist = []
                for key in attempt["Keys"]:
                    templist.append(key["Fight"])
                    templist.append(key["Dwell"])
                templist.append(attempt["Metadata"][0]["Total Time"])

                data.append(templist)
                labels.append(0)

data = np.asarray(data)
labels = tf.one_hot(labels,2)


In [16]:
#normalize data input, feature scaling for better learning
#theres probably a better way to do this but it works
data = data.astype(float)
for i in range(len(data)):
    data[i] = (data[i] - np.mean(data[i])) / np.std(data[i])
    
#normThis = []
#separate same values into separate lists to make math better
#for j in range(len(data[0])):
#    normThis.append([i[0] for i in data])
    
#print normThis[:2]
#do the standardization
#for i in range(len(normThis)):
#    normThis[i] = (normThis[i] - np.mean(normThis[i])) / np.std(normThis[i])

#print normThis[:2]

    

In [19]:
# Parameters
learning_rate = 0.2
training_runs = 4000


# tf Graph Input example from MNIST
#x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
#y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# tf Graph Input for PIN
x = tf.placeholder(tf.float32, [None, 9]) # each entry has 
y = tf.placeholder(tf.float32, [None, 2]) # one hot vector should be the output

# Set model weights
W1 = tf.Variable(tf.random_normal([9, 2]))
b1 = tf.Variable(tf.random_normal([2]))
#W1 = tf.Variable(tf.random_normal([9, 20]))
#b1 = tf.Variable(tf.random_normal([20]))
#W2 = tf.Variable(tf.zeros([20, 2]))
#b2 = tf.Variable(tf.zeros([2]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W1) + b1)
#z = tf.nn.softmax(tf.matmul(x, W1) + b1)
#pred = tf.nn.softmax(tf.matmul(z, W2) + b2)

# Minimize error using cross entropy
#cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
cost = -tf.reduce_sum(y*tf.log(tf.clip_by_value(pred,1e-10,1.0)))

# Gradient Descent
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [22]:
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

#turn labels back into an np array, needed session to be started before running
l = sess.run(labels)

#Create a saver object which will save all the variables
#saved_result= tf.Variable(data, name="saved_result")
#do_save=tf.assign(saved_result,pred)
#tf.train.write_graph(sess.graph_def, "/tmp/load", "test.pb", False) #proto

# Training cycle
print "predictions:", tf.argmax(pred.eval(feed_dict={x:data[:15]}),1).eval()
print pred.eval(feed_dict={x:data[:15]})
for i in range(training_runs):
    # Fit training using json data
    _, error = sess.run([optimizer, cost], {x:data[:15], y:l[:15]})

    # Print error every once and a while to see whats going on
    if (i % 10) == 0:
        #print sess.run(W1)
        #print sess.run(b1)
        print error

print "All done!"
print "predictions:", tf.argmax(pred.eval(feed_dict={x:data[:15]}),1).eval()
print pred.eval(feed_dict={x:data[:15]})
print "actual: ", l[:15]


predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[ 0.9876008   0.0123992 ]
 [ 0.9906261   0.00937387]
 [ 0.99061781  0.00938217]
 [ 0.9908765   0.00912342]
 [ 0.99108022  0.00891973]
 [ 0.9873749   0.0126251 ]
 [ 0.99072582  0.00927427]
 [ 0.99016541  0.00983457]
 [ 0.99102497  0.00897501]
 [ 0.99106395  0.00893609]
 [ 0.98929977  0.01070019]
 [ 0.99022722  0.00977273]
 [ 0.99045092  0.00954907]
 [ 0.99042612  0.00957393]
 [ 0.99084407  0.00915587]]
23.2584
11.7536
10.5326
9.88293
9.42507
9.3422
9.32071
9.29141
9.27024
9.25555
9.24026
9.22441
9.20806
9.19123
9.17393
9.15619
9.13804
9.11953
9.10069
9.08155
9.06215
9.04251
9.02267
9.00265
8.98248
8.96218
8.94178
8.9213
8.90075
8.88017
8.85956
8.83894
8.81832
8.79774
8.77718
8.75668
8.73623
8.71586
8.69556
8.67536
8.65525
8.63524
8.61534
8.59556
8.57589
8.55634
8.53692
8.51763
8.49846
8.47943
8.46052
8.44175
8.4231
8.40459
8.3862
8.36794
8.34981
8.33179
8.3139
8.29613
8.27847
8.26093
8.24349
8.22616
8.20894
8.19181
8.17478
8.15784
8.14099
8.

In [24]:
# now set the data:
saver = tf.train.Saver()
tf.train.write_graph(sess.graph_def, ".", "test.pb", False) #proto
saver.save(sess,"test.data")

'test.data'

In [30]:
# Launch the graph
# not using, this is just a reference
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(10):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/200)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(200)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % 10 == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)
            print sess.run(b1)

    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print "Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})

x: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
y: [[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]


ValueError: Cannot feed value of shape (5, 784) for Tensor u'Placeholder_16:0', which has shape '(?, 9)'